In [1]:
import argparse
import numpy as np

from dlgo.encoders import get_encoder_by_name
from dlgo import goboard
from dlgo.mcts import mcts
from dlgo.utils import print_board, print_move

In [2]:
def generate_game(board_size, rounds, max_moves, temperature):
    '''Генератор игровых данных методом Монте-Карло'''
    
    #boards - закодированное состояние доски
    #moves - закодированные ходы
    boards, moves = [], []
    
    #инициализация кодировщика OnePlaneEncoder и размер доски
    encoder = get_encoder_by_name('oneplane', board_size)
    
    #создание новой игры
    game = goboard.GameState.new_game(board_size)
    
    #в качестве бота будет агент поиска по дереву методом Монте-Карло
    #с указанием кол.раундов и температуры
    bot = mcts.MCTSAgent(rounds, temperature)
    
    num_moves = 0
    while not game.is_over():
        print_board(game.board)
        move = bot.select_move(game) #след.ход выбирается ботом
        if move.is_play:
            #закодированные данные о состоянии доски добавляются в boards
            boards.append(encoder.encode(game))
            
            #след.ход закодированный методом унитарного кодирования...
            move_one_hot = np.zeros(encoder.num_points())
            move_one_hot[encoder.encode_point(move.point)] = 1
            moves.append(move_one_hot) #...добавляются в moves
        
        print_move(game.next_player, move)
        game = game.apply_move(move) #ход бота применяется к доске
        num_moves += 1
        if num_moves > max_moves:
            #цикл повторяется, пока не достигнуто max кол.ходов
            break
    
    return np.array(boards), np.array(moves)

In [ ]:
#python generate_mcts_games.py -n 20 --board-out features.npy --move-out labels.npy
_a_board_size = 9
_a_rounds = 1000
_a_max_moves = 60
_a_temperature = 0.8
_a_num_games = 20
_a_board_out = 'features.npy'
_a_move_out = 'labels.npy'


xs = []
ys = []

for i in range(_a_num_games):
    print('Generating game %d/%d...' % (i+1, _a_num_games))
    #генерирование игровых данных
    x, y = generate_game(_a_board_size, _a_rounds, _a_max_moves, _a_temperature)
    xs.append(x)
    ys.append(y)
    
#после генерации всех игр производится конкатенация признаков и меток
x = np.concatenate(xs)
y = np.concatenate(ys)
    
#данные о признаках и метках сохраняем в отдельные файлы
np.save(_a_board_out, x)
np.save(_a_move_out, y)

Geberating game 1/20...
 9  .  .  .  .  .  .  .  .  . 
 8  .  .  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  .  .  . 
 6  .  .  .  .  .  .  .  .  . 
 5  .  .  .  .  .  .  .  .  . 
 4  .  .  .  .  .  .  .  .  . 
 3  .  .  .  .  .  .  .  .  . 
 2  .  .  .  .  .  .  .  .  . 
 1  .  .  .  .  .  .  .  .  . 
    A  B  C  D  E  F  G  H  J
